<a href="https://colab.research.google.com/github/33quitykubby/python_test/blob/feature%2Ftest/%E5%89%8D%E5%87%A6%E7%90%862E_%E5%AD%A6%E7%BF%92%E3%83%87%E3%83%BC%E3%82%BF%E3%83%AC%E3%83%A2%E3%83%B3%E5%88%87%E3%82%8A%E5%87%BA%E3%81%97_90%E5%BA%A6%E8%AA%BF%E6%95%B4_%E9%BB%92%E8%83%8C%E6%99%AF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python/OpenCVで画像内オブジェクトの輪郭抽出をする

https://watlab-blog.com/2020/03/19/find-contours/

In [ ]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection
import tensorflow as tf
import random
import pandas as pd
import datetime

import cv2

STANDARD_SEED = 0
NP_SEED = 0
TF_SEED = 0 

np.random.seed(STANDARD_SEED)
random.seed(NP_SEED)
tf.random.set_seed(TF_SEED)

# 目次
1. 切り取り関数
2. 学習データの切り取り
3. テストデータの切り取り

# 1.パス定義

In [ ]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/SIGNATE_LEMON/new_data/'

TRAIN_FILE_PATH = BASE_PATH + 'train_div/'
# TRAIN_FILE_PATH = BASE_PATH + 'train_temp/' # テスト用（うまくいかないやつ）

AUGMENT_TAIN_FILE_PATH = BASE_PATH + 'train_crop_4/'
# AUGMENT_TAIN_FILE_PATH = TRAIN_FILE_PATH # テスト用（うまくきかないやつ)

TEST_FILE_PATH = BASE_PATH + 'test_images/'

AUGMENT_TEST_FILE_PATH = BASE_PATH + 'test_crop_4/test_images/'

classes = ['0_Exellent', '1_Good', '2_ForProcess', '3_NonRegular']

# 1.1.ディレクトリ作成

In [ ]:
try:
    for i in range(len(classes)):
      dir_name = AUGMENT_TAIN_FILE_PATH + classes[i]
      print(dir_name)
      os.makedirs(dir_name)
except FileExistsError:
    pass

/content/drive/MyDrive/Colab Notebooks/SIGNATE_LEMON/new_data/train_crop_4/0_Exellent


# 2.ヒストグラム調整（未使用）
Contrast limited adaptive histogram equalization(CLAHE)を適用

PIL でカラー画像を Equalize(ヒストグラム平坦化)する際の注意  
https://qiita.com/yoya/items/a11085f90f555b887cf6

PIL(pillow)画像をnumpyにしたり戻したり  
https://chiyoh.hatenablog.com/entry/2019/05/04/145639

In [ ]:
from PIL import Image, ImageOps
import sys

def HistConvert(input_image):

  img_yuv = cv2.cvtColor(input_image, cv2.COLOR_BGR2YUV)

  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])

  dat_RGB = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

  return dat_RGB

# 3.切り取り関数

In [ ]:
import cv2
from matplotlib import pyplot as plt

def border_test (file, threshold):

  # カラー画像の読み込み
  img = cv2.imread(file, 1)

  # ヒストグラム平坦化
  # うまく切り出せてないのでやめ
  # img_histed = HistConvert(img)
  
  #リサイズ
  img_resized = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))

  # とっておく
  img_input = img_resized.copy()

  # グレースケール化
  img_gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
  
  # 単純二値化
  print("threshold = ", threshold)

  if ( threshold == 0):
    param = cv2.THRESH_OTSU
  else:
    param = cv2.THRESH_BINARY

  ret, img_binary = cv2.threshold(img_gray,
                                  threshold, 
                                  255,
                                  # cv2.THRESH_BINARY)
                                  param)

  # 輪郭抽出
  contours, hierarchy = cv2.findContours(img_binary,
                                        cv2.RETR_LIST,
                                        # cv2.CHAIN_APPROX_NONE)
                                        cv2.CHAIN_APPROX_SIMPLE)

  # print(contours)
  # print(contours[0])
  # print(contours[1])
  # print(contours[2])
  # print(hierarchy)
  # print(type(contours))
  # print(len(contours))

  pre_area = 0
  index = 0
  for i in range(0, len(contours)-1): #最後は画像全体になるので-1に
    cnt = contours[i]
    area = cv2.contourArea(cnt)
    x,y,w,h = cv2.boundingRect(cnt)
    # print(area,x,y,w,h)
    if area > pre_area:
      pre_area = area
      index = i
      (pre_x, pre_y, pre_w, pre_h) = (x, y, w, h)

  print("max={}:({},{}) w={},h={}".format(pre_area,pre_x,pre_y,pre_w,pre_h))

  # 輪郭を元画像に描画
  # img_contour = cv2.drawContours(img_resized, contours[:len(contours)-2], -1, (0, 255, 0), 5)
  # img_contour = cv2.drawContours(img_resized, contours[index], -1, (0, 255, 0), 5)
  img_contour = cv2.drawContours(img_resized, contours[index], -1, (0, 0, 0), 5,  lineType = cv2.LINE_AA)

  ###############################################################################################
  #　黒背景化
  ###############################################################################################
  
  border_len = len(contours[index])

  cv2.fillConvexPoly(img_contour, points =contours[index], color=(255,255,255))

  temp_contour = np.floor(img_contour/255) # マスク
  # print(temp_contour)
  # print(type(temp_contour))
  # print(np.shape(temp_contour))

  img_contour2 = temp_contour * img_input # マスクをかけて背景だけ黒に
  img_contour2 = img_contour2.astype(np.uint8) # 型変換
  print("===========================================================")
  # print("img_contour:type={},shape={},dtype={}".format(type(img_contour),np.shape(img_contour),img_contour.dtype))
  # # print(img_contour)
  # print("===========================================================")
  # print("img_contour2:type={},shape={},dtype={}".format(type(img_contour2),np.shape(img_contour2),img_contour2.dtype))
  # print(img_contour2)
  print(border_len)
  print("===========================================================")

  # ここから画像描画
  fig = plt.figure()
  ax1 = fig.add_subplot(111)
  ax1.imshow(cv2.cvtColor(img_contour2, cv2.COLOR_BGR2RGB))
  ax1.axis('off')
  plt.show()
  plt.close()

  return (pre_x,pre_y,pre_w,pre_h,ret,border_len,img_contour2)

# 4.ガウシアンフィルタ（未使用）  
https://algorithm.joho.info/programming/python/opencv-gaussian-filter-py/

In [ ]:
import cv2
from matplotlib import pyplot as plt

def gaussian (file):

    image = cv2.imread(file)

    height_image, width_image, c_image = image.shape

    # ksize_x = int(width_image * 0.1)
    # ksize_y = int(height_image * 0.1)
    ksize_x = 5
    ksize_y = 5

    print("ksize={},{}".format(ksize_x, ksize_y))

    sigma = 1.5

    gauss_out = cv2.GaussianBlur(image, ksize=(ksize_x, ksize_y), sigmaX=sigma)

    return gauss_out


# 5.学習データの輪郭切り取り

In [ ]:
#現在時刻
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print(now)

2021-02-17 20:25:15.918769+09:00


In [ ]:
# classes = ['0_Exellent', '1_Good', '2_ForProcess', '3_NonRegular']

num_classes = len(classes)
IMAGE_SIZE = 600 #EfficientNet-B7


X = [] # image file
Y = [] # correct label
# TRAIN_FILE_PATH = BASE_PATH + 'train_div/'

# AUGMENT_TAIN_FILE_PATH = BASE_PATH + 'train_crop_2/'

for index, classlabel in enumerate(classes):
    photo_dir = TRAIN_FILE_PATH + classlabel
    print("photo_dir",photo_dir)
    files = sorted(glob.glob(photo_dir + '/*.jpg'))
    ii = 0
    for ii, file in enumerate(files):
        print("{}:file={}".format(ii,file))

        # 3段階でthreshを変更
        # 幅、高さが201から599の間ならPass
        for  i in range(5):
          # if i == 0:
          #   thresh = 0
          # elif i == 1:
          #   thresh = 60
          # elif i == 2:
          #   thresh = 90
          # else:
          #   thresh = 120
          if i == 0:
            thresh = 60
          elif i == 1:
            thresh = 90
          elif i == 2:
            thresh = 120
          elif i == 3:
            thresh = 75
          elif i == 4:
            thresh = 105
          else:
            thresh = 0
          (xx, yy, ww, hh, th, border_len, img_black_back) =border_test(file, thresh)
          if ((200 < ww and ww < 510) and (200 < hh and hh < 510)):
            if (border_len < 650):
              print("OK!")
              break

        # image = cv2.imread(file,0)
        # image = cv2.imread(file)

        # ヒストグラム平坦化
        # img_histed = HistConvert(image)

        # img_resized = image
        img_resized = cv2.resize(img_black_back, (IMAGE_SIZE, IMAGE_SIZE)) #リサイズ

        ##########　拡大 ############

        # 長辺に合わせて正方形に切り出し

        OFFSET_MERGIN = 15 # 切り出しの周囲に付与するマージン

        if ww > hh:
          print ("ww > hh")
          crop_length = ww + OFFSET_MERGIN * 2 #切り出しの一辺
          offset_y = yy - ( crop_length - hh ) * 0.5
          offset_x = xx - OFFSET_MERGIN
          offset_yy = offset_y + crop_length
          offset_xx = offset_x + crop_length
        else:
          print ("ww <= hh")
          crop_length = hh + OFFSET_MERGIN * 2 #切り出しの一辺
          offset_y = yy - OFFSET_MERGIN
          offset_x = xx - ( crop_length - ww ) * 0.5
          offset_yy = offset_y + crop_length
          offset_xx = offset_x + crop_length


        #本当は反対側のオフセットも再計算するべきだが、ひとまずこれで
        if offset_x < 0 :
          print("offset_x set to 0")
          offset_x = 0
        if offset_y < 0 :
          print("offset_y set to 0")
          offset_y = 0
        if offset_xx > IMAGE_SIZE:
          print("offset_xx set to IMAGE SIZE")
          offset_xx = IMAGE_SIZE
        if offset_yy > IMAGE_SIZE:
          print("offset_yy set to IMAGE SIZE")
          offset_yy = IMAGE_SIZE

        offset_x = int(offset_x) #横方向　左端から切り出し開始までのピクセル数
        offset_y = int(offset_y) #縦方向　上端から切り出し開始までのピクセル数

        offset_xx = int(offset_xx) #横方向　左端から切り出されるところまでのピクセル数
        offset_yy = int(offset_yy) #縦方向　上端から切り出されるところまでのピクセル数

        print("x={}, xx={}, y={}, yy={}".format(offset_x, offset_xx, offset_y, offset_yy))
        
        # 切り出し
        im_cropped = img_resized[offset_y:offset_yy, offset_x:offset_xx, :]

        write_image = im_cropped

        # ファイル出力
        # ファイル名だけ抽出
        # temp_file = file.lstrip(photo_dir+'/')#ファイルパス削除にlstripは使えない（一致した文字から先が消える）
        temp_file = file.replace(photo_dir+'/', "")
        temp_file = temp_file.replace(".jpg", "_" + str(th) + ".jpg") #ファイル名に切り出しのthを入れておく(うまく切り出せてない奴の確認のため)
        print("file=",temp_file)
        # 
        # outfile_name = AUGMENT_TAIN_FILE_PATH + classlabel + '/temp/m_' + temp_file # テスト用（うまくきかないやつ)
        outfile_name = AUGMENT_TAIN_FILE_PATH + classlabel + '/' + temp_file
        print("output=",outfile_name)
        cv2.imwrite(outfile_name, write_image)

        ###############################################
        # 90度回転 
        ###############################################
        write_image2 = np.rot90(write_image)

        # ファイル出力
        # ファイル名だけ抽出
        temp_file = temp_file.replace(".jpg", "_r.jpg")
        print("90dig rotetion file=",temp_file)
        # 
        # outfile_name = AUGMENT_TAIN_FILE_PATH + classlabel + '/temp/m_' + temp_file # テスト用（うまくきかないやつ)
        outfile_name = AUGMENT_TAIN_FILE_PATH + classlabel + '/' + temp_file
        print("90dig rotetion output=",outfile_name)

        cv2.imwrite(outfile_name, write_image2)

        # if ii > 2 :
        #   print("breaking!!!!!!")
        #   break

print("########### end ###########")

In [ ]:
#現在時刻
now2 = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print(now2)
print(now2-now)

2021-02-17 07:28:47.470385+09:00
0:00:00.155565


# 6.テストデータの切り取り

In [ ]:
#現在時刻
now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print(now)

2021-02-18 08:23:13.832919+09:00


In [ ]:
IMAGE_SIZE = 600 #EfficientNet-B7


X = [] # image file
Y = [] # correct label

photo_dir = TEST_FILE_PATH
print("photo_dir",photo_dir)
files = sorted(glob.glob(photo_dir + '/*.jpg'))
for ii, file in enumerate(files):
    print("{}:file={}".format(ii,file))

    for  i in range(5):
      # if i == 0:
      #   thresh = 0
      # elif i == 1:
      #   thresh = 60
      # elif i == 2:
      #   thresh = 90
      # else:
      #   thresh = 120
      if i == 0:
        thresh = 60
      elif i == 1:
        thresh = 90
      elif i == 2:
        thresh = 120
      elif i == 3:
        thresh = 75
      elif i == 4:
        thresh = 105
      else:
        thresh = 0
      (xx, yy, ww, hh, th, border_len, img_black_back) =border_test(file, thresh)
      if ((200 < ww and ww < 510) and (200 < hh and hh < 510)):
        if (border_len < 650):
          print("OK!")
          break


    # image = cv2.imread(file,0)
    # image = cv2.imread(file)

    # ヒストグラム平坦化
    # img_histed = HistConvert(image)

    # img_resized = image
    img_resized = cv2.resize(img_black_back, (IMAGE_SIZE, IMAGE_SIZE)) #リサイズ

    ##########　拡大 ############

    # 長辺に合わせて正方形に切り出し

    OFFSET_MERGIN = 15 # 切り出しの周囲に付与するマージン

    if ww > hh:
      print ("ww > hh")
      crop_length = ww + OFFSET_MERGIN * 2 #切り出しの一辺
      offset_y = yy - ( crop_length - hh ) * 0.5
      offset_x = xx - OFFSET_MERGIN
      offset_yy = offset_y + crop_length
      offset_xx = offset_x + crop_length
    else:
      print ("ww <= hh")
      crop_length = hh + OFFSET_MERGIN * 2 #切り出しの一辺
      offset_y = yy - OFFSET_MERGIN
      offset_x = xx - ( crop_length - ww ) * 0.5
      offset_yy = offset_y + crop_length
      offset_xx = offset_x + crop_length


    #本当は反対側のオフセットも再計算するべきだが、ひとまずこれで
    if offset_x < 0 :
      print("offset_x set to 0")
      offset_x = 0
    if offset_y < 0 :
      print("offset_y set to 0")
      offset_y = 0
    if offset_xx > IMAGE_SIZE:
      print("offset_xx set to IMAGE SIZE")
      offset_xx = IMAGE_SIZE
    if offset_yy > IMAGE_SIZE:
      print("offset_yy set to IMAGE SIZE")
      offset_yy = IMAGE_SIZE

    offset_x = int(offset_x) #横方向　左端から切り出し開始までのピクセル数
    offset_y = int(offset_y) #縦方向　上端から切り出し開始までのピクセル数

    offset_xx = int(offset_xx) #横方向　左端から切り出されるところまでのピクセル数
    offset_yy = int(offset_yy) #縦方向　上端から切り出されるところまでのピクセル数

    print("x={}, xx={}, y={}, yy={}".format(offset_x, offset_xx, offset_y, offset_yy))
    
    # 切り出し
    im_cropped = img_resized[offset_y:offset_yy, offset_x:offset_xx, :]

    write_image = im_cropped

    # ファイル出力
    # ファイル名だけ抽出
    # temp_file = file.lstrip(photo_dir+'/')#ファイルパス削除にlstripは使えない（一致した文字から先が消える）
    temp_file = file.replace(photo_dir, "")
    # temp_file = temp_file.replace(".jpg", "_" + str(th) + ".jpg") #ファイル名に切り出しのthを入れておく(うまく切り出せてない奴の確認のため)
    print("file={} / th={}".format(temp_file,th))
    # 
    outfile_name = AUGMENT_TEST_FILE_PATH + temp_file
    print("output=",outfile_name)
    cv2.imwrite(outfile_name, write_image)


    # if ii > 2 :
    #   print("breaking!!!!!!")
    #   break

print("########### end ###########")

In [ ]:
#現在時刻
now2 = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print(now2)
print(now2-now)

2021-02-18 08:23:39.174019+09:00
0:00:25.341100
